In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Partiotion').master('local[5]').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/15 20:18:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.range(0,20)
df.rdd.getNumPartitions()


5

In [4]:
rdd1 = spark.sparkContext.parallelize((0,20), 10)
rdd1.getNumPartitions()

10

### File in partitions

In [5]:
file_RDD = spark.sparkContext.textFile('./data/deporte.csv', 10)
file_RDD.getNumPartitions()

10

In [6]:
file_RDD.saveAsTextFile('./data/outputtext')

In [7]:
! ls ./data/outputtext/

_SUCCESS    part-00001	part-00003  part-00005	part-00007  part-00009
part-00000  part-00002	part-00004  part-00006	part-00008


In [8]:
! cat ./data/outputtext/_SUCCESS

In [9]:
! head -n 5 ./data/outputtext/part-00003

21,Hockey
22,Rowing
23,Bobsleigh
24,Fencing
25,Equestrianism


### Merge partitions

In [10]:
rdd = spark.sparkContext.wholeTextFiles('./data/outputtext/*')
rdd.take(5)

[('file:/workspace/data/outputtext/part-00000',
  'deporte_id,deporte\n1,Basketball\n2,Judo\n3,Football\n4,Tug-Of-War\n5,Speed Skating\n6,Cross Country Skiing\n'),
 ('file:/workspace/data/outputtext/part-00001',
  '7,Athletics\n8,Ice Hockey\n9,Swimming\n10,Badminton\n11,Sailing\n12,Biathlon\n13,Gymnastics\n14,Art Competitions\n'),
 ('file:/workspace/data/outputtext/part-00002',
  '15,Alpine Skiing\n16,Handball\n17,Weightlifting\n18,Wrestling\n19,Luge\n20,Water Polo\n'),
 ('file:/workspace/data/outputtext/part-00003',
  '21,Hockey\n22,Rowing\n23,Bobsleigh\n24,Fencing\n25,Equestrianism\n26,Shooting\n27,Boxing\n28,Taekwondo\n'),
 ('file:/workspace/data/outputtext/part-00004',
  '29,Cycling\n30,Diving\n31,Canoeing\n32,Tennis\n33,Modern Pentathlon\n34,Figure Skating\n35,Golf\n')]

In [11]:
l = rdd.mapValues(lambda x: x.split()).collect()
l

[('file:/workspace/data/outputtext/part-00000',
  ['deporte_id,deporte',
   '1,Basketball',
   '2,Judo',
   '3,Football',
   '4,Tug-Of-War',
   '5,Speed',
   'Skating',
   '6,Cross',
   'Country',
   'Skiing']),
 ('file:/workspace/data/outputtext/part-00001',
  ['7,Athletics',
   '8,Ice',
   'Hockey',
   '9,Swimming',
   '10,Badminton',
   '11,Sailing',
   '12,Biathlon',
   '13,Gymnastics',
   '14,Art',
   'Competitions']),
 ('file:/workspace/data/outputtext/part-00002',
  ['15,Alpine',
   'Skiing',
   '16,Handball',
   '17,Weightlifting',
   '18,Wrestling',
   '19,Luge',
   '20,Water',
   'Polo']),
 ('file:/workspace/data/outputtext/part-00003',
  ['21,Hockey',
   '22,Rowing',
   '23,Bobsleigh',
   '24,Fencing',
   '25,Equestrianism',
   '26,Shooting',
   '27,Boxing',
   '28,Taekwondo']),
 ('file:/workspace/data/outputtext/part-00004',
  ['29,Cycling',
   '30,Diving',
   '31,Canoeing',
   '32,Tennis',
   '33,Modern',
   'Pentathlon',
   '34,Figure',
   'Skating',
   '35,Golf']),
 ('fi

In [13]:
l2 = [i[0] for i in l]
l2.sort()
l2

['file:/workspace/data/outputtext/part-00000',
 'file:/workspace/data/outputtext/part-00001',
 'file:/workspace/data/outputtext/part-00002',
 'file:/workspace/data/outputtext/part-00003',
 'file:/workspace/data/outputtext/part-00004',
 'file:/workspace/data/outputtext/part-00005',
 'file:/workspace/data/outputtext/part-00006',
 'file:/workspace/data/outputtext/part-00007',
 'file:/workspace/data/outputtext/part-00008',
 'file:/workspace/data/outputtext/part-00009']

In [15]:
rdd2 = spark.sparkContext.textFile(','.join(l2)).map( lambda x: x.split(','))
rdd2.take(3)

[['deporte_id', 'deporte'], ['1', 'Basketball'], ['2', 'Judo']]